In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
from nipype.interfaces import brainsuite
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from shutil import copyfile
from pandas import Series
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()

In [15]:
#Set variables
subject_list = ['06-T1']#, '08-T1', '10-T1', '11-T1']#,'02-T1', '03-T1']
home = expanduser('~')
fpath = join(home, 'Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1')
workflow_dir = fpath + '/workflow_qT1'
data_dir = fpath + '/DataDir_qT1'
if fpath == '/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1':
    workflow_dir = fpath + '/workflow_lucinda'
elif fpath == '/Users/myelin/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1':
    workflow_dir = fpath + '/workflow_myelin'

In [16]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(dti=fpath+'/{subject_id}/raw/dti.nii.gz',
               t1=fpath+'/{subject_id}/raw/T1_acpc.nii.gz',
               qt1=fpath+'/{subject_id}/qt1_T1fit_final.nii.gz',
               unc_maskL = fpath+'/{subject_id}/Left_Uncinate.nii.gz',
               unc_maskR = fpath+'/{subject_id}/Right_Uncinate.nii.gz')

# 
#             bval=fpath+'/{subject_id}/dti.bval',
#             bvec=fpath+'/{subject_id}/dti.bvec'

sf = Node(SelectFiles(template, 
                      base_directory = fpath),
          name = 'sf')


In [17]:
#Reorient DTI data to standard orientation
reorient = Node(fsl.Reorient2Std(),
                name = 'reorient')

In [18]:
#FSL split - extract DTI b0 image
split = Node(fsl.ExtractROI(t_min = 1,
                            t_size = 1,
                           roi_file = 'b0.nii.gz'),
             name = 'split')

In [19]:
#Skullstrip T1w image
stripT1 = Node(brainsuite.Bse(outputDetailedBrainMask = 'bs_detailedbrainmask.nii.gz',
                              outputMRIVolume= 'bs_skullstripped_anat.nii.gz',
                              outputMaskFile = 'bs_maskfile.nii.gz'),
             name = 'stripT1')
# stripT1.inputs.inputMRIFile = '/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/06-T1/raw/T1_acpc.nii.gz'
# stripT1.run()

In [20]:
# #Skullstrip T1w image
# stripT1 = Node(fsl.BET(frac = 0.2,
#                      out_file = 'skullstripped_T1.nii.gz',
#                      output_type = 'NIFTI_GZ'),
#              name = 'stripT1')

In [21]:
#Skullstrip b0 image
strip = Node(fsl.BET(frac = 0.2,
                     out_file = 'skullstripped_b0.nii.gz',
                     output_type = 'NIFTI_GZ'),
             name = 'strip')

In [22]:
#Register DTI b0 to T1w
register_b0 = Node(fsl.FLIRT(out_file = 'registered_b0.nii.gz',
                          output_type = 'NIFTI_GZ',
                          out_matrix_file = 'flirt_transform_dti.mat'),
                name = 'register_b0')

In [23]:
#Register qT1 to T1w
register_qt1 = Node(fsl.FLIRT(out_file = 'registered_qT1.nii.gz',
                            output_type = 'NIFTI_GZ',
                            out_matrix_file = 'flirt_transform_qt1.mat'),
                  name = 'register_qt1')

In [24]:
# #Apply transform generated from registration to entire DTI file
applyxfm = Node(fsl.ApplyXFM(out_file = 'dti_applied_xfm.nii.gz',
                             apply_xfm = True,
                             output_type = 'NIFTI_GZ'),
                name = 'applyxfm')

In [25]:
#Extract voxel values from qT1 beneath overlaid mask - Left Uncinate
get_voxL = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftUncinate.txt'),
               name = 'get_voxL')

In [26]:
#Extract voxel values from qT1 beneath overlaid mask - Right Uncinate
get_voxR = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightUncinate.txt'),
               name = 'get_voxR')

In [27]:
qt1proc_flow = Workflow(name = 'qt1proc_flow')
qt1proc_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                      (sf, reorient, [('dti', 'in_file')]),
                      (reorient, datasink, [('out_file', '1_dti_raw_reoriented')]),
                      (sf, stripT1, [('t1', 'inputMRIFile')]),
                      (stripT1, datasink, [('outputMRIVolume', '2a_skullstripped_t1s')]),
                      (reorient, split, [('out_file', 'in_file')]),
                      (split, strip, [('roi_file', 'in_file')]),
                      (strip, datasink, [('out_file', '2b_skullstripped_b0')]),
                      (strip, register_b0, [('out_file', 'in_file')]),
                      (stripT1, register_b0, [('outputMRIVolume', 'reference')]),
                      (register_b0, datasink, [('out_file', '3_registered_files'),
                                            ('out_matrix_file','3_registered_files.@par' )]),
                      (sf, register_qt1, [('qt1', 'in_file')]),
                      (register_b0, register_qt1, [('out_file', 'reference')]),
                      (register_qt1, datasink, [('out_file', '3_registered_files.@par.@par'),
                                            ('out_matrix_file','3_registered_files.@par.@par.@par' )]),
                      (register_b0, applyxfm, [('out_matrix_file', 'in_matrix_file')]),
                      (reorient, applyxfm, [('out_file', 'in_file')]),
                      (register_b0, applyxfm, [('out_file', 'reference')]),
                      (applyxfm, datasink, [('out_file', '4_applied_transform')]),
                      (register_qt1, get_voxL, [('out_file', 'in_file')]),
                      (sf, get_voxL, [('unc_maskL', 'mask')]),
                      (get_voxL, datasink, [('out_file', '5_Extracted_Stats')]),
                      (register_qt1, get_voxR, [('out_file', 'in_file')]),
                      (sf, get_voxR, [('unc_maskR', 'mask')]),
                      (get_voxR, datasink, [('out_file', '5_Extracted_Stats.@par')])
                     ])
qt1proc_flow.base_dir = workflow_dir
qt1proc_flow.write_graph(graph2use = 'flat')
preproc = qt1proc_flow.run('MultiProc', plugin_args={'n_procs': 4})

190210-12:59:21,53 nipype.workflow DEBUG:
	 (qt1proc_flow.infosource, qt1proc_flow.sf): No edge data
190210-12:59:21,55 nipype.workflow DEBUG:
	 (qt1proc_flow.infosource, qt1proc_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190210-12:59:21,56 nipype.workflow DEBUG:
	 (qt1proc_flow.sf, qt1proc_flow.reorient): No edge data
190210-12:59:21,58 nipype.workflow DEBUG:
	 (qt1proc_flow.sf, qt1proc_flow.reorient): new edge data: {'connect': [('dti', 'in_file')]}
190210-12:59:21,59 nipype.workflow DEBUG:
	 (qt1proc_flow.reorient, qt1proc_flow.datasink): No edge data
190210-12:59:21,61 nipype.workflow DEBUG:
	 (qt1proc_flow.reorient, qt1proc_flow.datasink): new edge data: {'connect': [('out_file', '1_dti_raw_reoriented')]}
190210-12:59:21,62 nipype.workflow DEBUG:
	 (qt1proc_flow.sf, qt1proc_flow.stripT1): No edge data
190210-12:59:21,64 nipype.workflow DEBUG:
	 (qt1proc_flow.sf, qt1proc_flow.stripT1): new edge data: {'connect': [('t1', 'inputMRIFile')]}
190210-12:59:21,65

190210-12:59:21,516 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.infosource
190210-12:59:21,518 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.sf
190210-12:59:21,519 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.stripT1
190210-12:59:21,521 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.reorient
190210-12:59:21,523 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.split
190210-12:59:21,525 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.strip
190210-12:59:21,526 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.register_b0
190210-12:59:21,528 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.applyxfm
190210-12:59:21,529 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.register_qt1
190210-12:59:21,531 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.get_voxR
190210-12:59:21,532 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.get_voxL
190210-12:59:21,533 nipype.workflow DEBUG:
	 processing node: qt1proc_flow.datas

190210-12:59:23,584 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 4/4.
190210-12:59:23,655 nipype.workflow DEBUG:
	 Allocating qt1proc_flow.stripT1 ID=1 (0.20GB, 1 threads). Free: 28.60GB, 3 threads.
190210-12:59:23,657 nipype.workflow DEBUG:
	 Setting node inputs
190210-12:59:23,659 nipype.workflow DEBUG:
	 input: inputMRIFile
190210-12:59:23,661 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/sf/result_sf.pklz
190210-12:59:23,663 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/sf/result_sf.pklz
190210-12:59:23,665 nipype.workflow DEBUG:
	 output: t1
190210-12:59:23,667 nipype.workflow DEBUG:
	 [Node] stripT1 - setting input inputMRIFile = /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/06-T1/raw/T1_acpc.nii.gz
19

190210-12:59:27,630 nipype.workflow DEBUG:
	 [Node] Up-to-date cache found for "qt1proc_flow.strip".
190210-12:59:27,632 nipype.workflow DEBUG:
	 Checking hash "qt1proc_flow.strip" locally: cached=True, updated=True.
190210-12:59:27,633 nipype.workflow DEBUG:
	 Skipping cached node qt1proc_flow.strip with ID 4.
190210-12:59:27,635 nipype.workflow INFO:
	 [Job 4] Cached (qt1proc_flow.strip).
190210-12:59:29,581 nipype.workflow DEBUG:
	 Progress: 11 jobs, 5/0/1 (done/running/ready), 0/6 (pending_tasks/waiting).
190210-12:59:29,582 nipype.workflow DEBUG:
	 Tasks currently running: 0. Pending: 0.
190210-12:59:29,621 nipype.workflow DEBUG:
	 Allocating qt1proc_flow.register_b0 ID=5 (0.20GB, 1 threads). Free: 28.60GB, 3 threads.
190210-12:59:29,622 nipype.workflow DEBUG:
	 Setting node inputs
190210-12:59:29,624 nipype.workflow DEBUG:
	 input: reference
190210-12:59:29,625 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1pr

190210-12:59:31,666 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/sf/result_sf.pklz
190210-12:59:31,668 nipype.workflow DEBUG:
	 output: qt1
190210-12:59:31,670 nipype.workflow DEBUG:
	 [Node] register_qt1 - setting input in_file = /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/06-T1/qt1_T1fit_final.nii.gz
190210-12:59:31,672 nipype.workflow DEBUG:
	 input: reference
190210-12:59:31,673 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/register_b0/result_register_b0.pklz
190210-12:59:31,675 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/register_b0/result_register_b0.pklz
190210-12:59:31,676 nipype.workflow DEBUG:
	 output: out_file
190210-12:59:31,678 nipype.workflow DEBUG:
	 [Node] register

...

190210-12:59:35,694 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/applyxfm/result_applyxfm.pklz
190210-12:59:35,696 nipype.workflow DEBUG:
	 output: out_file
190210-12:59:35,697 nipype.workflow DEBUG:
	 [Node] datasink - setting input 4_applied_transform = /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/applyxfm/dti_applied_xfm.nii.gz
190210-12:59:35,698 nipype.workflow DEBUG:
	 input: 5_Extracted_Stats
190210-12:59:35,700 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/get_voxL/result_get_voxL.pklz
190210-12:59:35,701 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/get_voxL/result_get_voxL.pklz
190210-12:59:35,703 nipype.workflow DEBUG:
	 outpu

190210-12:59:35,721 nipype.workflow DEBUG:
	 Checking hash "qt1proc_flow.datasink" locally: cached=True, updated=False.
190210-12:59:35,724 nipype.workflow DEBUG:
	 [MultiProc] Submitted task qt1proc_flow.datasink (taskid=4).
190210-12:59:35,727 nipype.workflow INFO:
	 [Node] Setting-up "qt1proc_flow.datasink" in "/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/datasink".
190210-12:59:35,734 nipype.workflow DEBUG:
	 [Node] Hashes: [('_outputs', [('1_dti_raw_reoriented', ('/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/reorient/dti_reoriented.nii.gz', '2cdf2ad618d27b095939f27e799f9520')), ('2a_skullstripped_t1s', ('/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/stripT1/bs_skullstripped_anat.nii.gz', '1c90593a761d03bb4efbf2f0670a349e')), ('2b_skullstripped_b0', ('/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/

	 [Node] Writing pre-exec report to "/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/datasink/_report/report.rst"
190210-12:59:35,767 nipype.workflow DEBUG:
	 copying files to wd [execute=True, linksonly=False]
190210-12:59:35,771 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
190210-12:59:35,776 nipype.interface DEBUG:
	 key: 1_dti_raw_reoriented files: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/reorient/dti_reoriented.nii.gz
190210-12:59:35,780 nipype.interface DEBUG:
	 copyfile: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/reorient/dti_reoriented.nii.gz /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/1_dti_raw_reoriented/_subject_id_06-T1/dti_reoriented.nii.gz
190210-12:59:35,783 nipype.utils DEBUG:
	 /Users/lucindasisk/Dropbox/SNAPLab/Data/T

	 /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/3_registered_files/_subject_id_06-T1/flirt_transform_dti.mat
190210-12:59:36,338 nipype.utils DEBUG:
	 File: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/3_registered_files/_subject_id_06-T1/flirt_transform_dti.mat already exists,21dde44e167d283e60916a5c93d7e22e, copy:1
190210-12:59:36,341 nipype.utils DEBUG:
	 File: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/3_registered_files/_subject_id_06-T1/flirt_transform_dti.mat already exists, not overwriting, copy:1
190210-12:59:36,344 nipype.interface DEBUG:
	 key: 3_registered_files.@par.@par files: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/register_qt1/registered_qT1.nii.gz
190210-12:59:36,347 nipype.interface DEBUG:
	 copyfile: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/workflow_lucinda/qt1proc_flow/_subject_id_06-T1/register_qt1/regis

	 File: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/5_Extracted_Stats/_subject_id_06-T1/ROI_voxel-contrasts_RightUncinate.txt already exists, not overwriting, copy:1
190210-12:59:38,519 nipype.workflow DEBUG:
	 Needed files: /Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/1_dti_raw_reoriented/_subject_id_06-T1/dti_reoriented.nii.gz;/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/2a_skullstripped_t1s/_subject_id_06-T1/bs_skullstripped_anat.nii.gz;/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/2b_skullstripped_b0/_subject_id_06-T1/skullstripped_b0.nii.gz;/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/3_registered_files/_subject_id_06-T1/registered_b0.nii.gz;/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/3_registered_files/_subject_id_06-T1/flirt_transform_dti.hdr;/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/3_registered